In [3]:
import pandas as pd
data_file_path = 'Dataset.csv'
data_read = pd.read_csv(data_file_path)
data_read.describe()

,Unnamed: 0,vote_count,vote_average,duration,revenue,popularity,id,financial_investment
count,4803.000000,4803.000000,4323.000000,4801.000000,4.755000e+03,4803.000000,4803.000000,4.419000e+03
mean,2401.000000,690.217989,6.090354,106.875859,8.231486e+07,21.492301,57165.484281,2.898466e+07
std,1386.651002,1234.585891,1.193315,22.611935,1.630872e+08,31.816650,88694.614033,4.065526e+07
min,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,5.000000,0.000000e+00
25%,1200.500000,54.000000,5.600000,94.000000,0.000000e+00,4.668070,9014.500000,7.500000e+05
50%,2401.000000,235.000000,6.200000,103.000000,1.918402e+07,12.921594,14629.000000,1.480000e+07
75%,3601.500000,737.000000,6.800000,118.000000,9.311911e+07,28.313505,58610.500000,4.000000e+07
max,4802.000000,13752.000000,10.000000,338.000000,2.787965e+09,875.581305,459488.000000,3.800000e+08
